In [28]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')


<h1 align="center">Synchrotron Tools</h1>

<h2 align="left">Select phase and EoS formalism</h2>

In [29]:
#!jupyter nbextension enable --py --sys-prefix widgetsnbextension

import ipywidgets as widgets
import numpy as np
import sys

from ipywidgets import Layout, HBox, VBox
from IPython.core.display import display
from scipy.optimize import fsolve

global P_max, phase, EoS
P_max, V0, EoS = 200, 40, 1

def phase_selector(phase):
    global V0, K, Kp, a0, b0, c0
    if phase == 2:
        dropdown2.value, dropdown2.disabled = 2, True 
        #Data from: Tange et al. (2012) 10.1029/2011JB008988 and Tange et al. (2009) 10.1029/2008JB005891
        V0, K, Kp, a0, b0, c0 = 162.373, 258.4, 4.1, 4.7769, 4.9284, 6.8972
        initialise()
    if phase == 3: 
        #Data from: Fiquet et al. (2002) American Mineralogist 87:1261-1265
        dropdown2.value, dropdown2.disabled = 2, True 
        V0, K, Kp, a0, b0, c0 = 279.2, 108, 5, 4.6377, 4.6377, 15.007
        initialise()
    if phase == 4:
        dropdown2.value, dropdown2.disabled = 1, False
    if phase == 5:
        #Data from: Dorogokupets & Dewaele (2007) 10.1080/08957950701659700
        dropdown2.value, dropdown2.disabled = 3, True 
        V0, K, Kp, a0, b0, c0 = 60.38, 277.3, 5.12, 3.9231, 3.9231, 3.9231
        initialise()
    if phase == 6:
        dropdown2.value, dropdown2.disabled = 1, False
    if phase == 7:
        #Data from: Dorogokupets & Dewaele (2007) 10.1080/08957950701659700
        dropdown2.value, dropdown2.disabled = 3, True 
        V0, K, Kp, a0, b0, c0 = 179.44, 23.83, 5.09, 5.6404, 5.6404, 5.6404
        initialise()
    if phase == 8:
        #Data from: Dorogokupets & Dewaele (2007) 10.1080/08957950701659700
        dropdown2.value, dropdown2.disabled = 3, True 
        V0, K, Kp, a0, b0, c0 = 40.73, 29.72, 5.14, 3.4407, 3.4407, 3.4407
        initialise()
                
def EoS_selector(EoS,phase):
    global V0, K, Kp, a0, b0, c0
    if phase == 4:
        #Data from: Wang et al. (2012) 10.1029/2011JB009100
        if EoS == 2:
            V0, K, Kp, a0, b0, c0 = 46.55, 294, 4.85, 4.66, 4.66, 2.8403
            initialise()
        if EoS == 3:
            V0, K, Kp, a0, b0, c0 = 46.55, 292, 5.01, 4.66, 4.66, 2.8403
            initialise()
        if EoS == 1:
            K_box.value, Kp_box.value, V0_box.value, VV_box.value, Va_box.value, Vb_box.value, Vc_box.value  = (''), (''), (''), (''), (''), (''), ('')
    if phase == 6:
        #Data from: Fei et al. (2012) 10.1073/pnas.0609013104
        if EoS == 2:
            V0, K, Kp, a0, b0, c0 = 67.85, 167, 5.77, 4.0786, 4.0786, 4.0786
            initialise()
        if EoS == 3:
            V0, K, Kp, a0, b0, c0 = 67.85, 167, 6.00, 4.0786, 4.0786, 4.0786
            initialise()
        if EoS == 1:
            K_box.value, Kp_box.value, V0_box.value, VV_box.value, Va_box.value, Vb_box.value, Vc_box.value  = (''), (''), (''), (''), (''), (''), ('')
            
def initialise():
    K_box.value = ("K = " + str(K) + " GPa")
    Kp_box.value = ("K' = " + str(Kp))
    V0_box.value = ("V₀ = " + str(V0) + " Å³")
    VV_box.value = ("V = " + str(V0) + " Å³")
    Va_box.value = ("a = " + str(a0) + " Å")
    Vb_box.value = ("b = " + str(b0) + " Å")
    Vc_box.value = ("c = " + str(c0) + " Å")
    P_slider.value = 0
        

def BM3(V,P):
    return P-((3/2)*K*(((V0/V)**(7/3))-((V0/V)**(5/3)))*(1-(3/4)*(4-Kp))*((V0/V)**(2/3)-1))

def Vinet(V,P):
    return P-(3*K*((1-((V/V0)**(1/3)))/((V/V0)**(2/3)))*np.exp((3/2)*(Kp-1)*(1-((V/V0)**(1/3)))))
            
dropdown1 = widgets.Dropdown(options = {'Choose': 1, 'MgSiO3 perovskite': 2, 'MgCO3 magnesite': 3, 'SiO2 stishovite': 4, 'Pt': 5, 'Au': 6, 'NaCl B1': 7, },description='Phase:',layout=Layout(width='47.5%'))
dropdown2 = widgets.Dropdown(options = {'Choose': 1, '3rd Order Birch-Murnaghan': 2, 'Vinet': 3},description='EoS:',layout=Layout(width='47.5%'))    

w1=widgets.interactive(phase_selector,phase=dropdown1)
w2=widgets.interactive(EoS_selector,EoS=dropdown2,phase=dropdown1)

K_box = widgets.Text(disabled=True,text_color='green',background_color='green',layout=Layout(width='12%'))
Kp_box = widgets.Text(disabled=True,text_color='green',background_color='green',layout=Layout(width='12%'))
V0_box = widgets.Text(disabled=True,text_color='green',background_color='green',layout=Layout(width='12%'))

VBox([HBox([dropdown1]), HBox([dropdown2, K_box, Kp_box, V0_box])])

<h2 align="left">Select EoS</h2>

<h2 align="left">TOOL 1: PV calculator</h2>

In [30]:
global P_max, phase, EoS, V0, K, Kp, a0, b0, c0
def V_calc(P):
        if EoS == 1 or 2:
            V = fsolve(lambda V,P: BM3(V,P), 1, args=(P), xtol=1e-5)
        if EoS == 3:
            V = fsolve(lambda V,P: Vinet(V,P), 1, args=(P), xtol=1e-5)
        VV_box.value = ("V = " + str(np.around(V,3))[1:-1] + " Å³")
        Va_box.value = ("a = " + str(np.around((V/V0)**(1/3)*a0,3))[1:-1] + " Å")
        Vb_box.value = ("b = " + str(np.around((V/V0)**(1/3)*b0,3))[1:-1] + " Å")
        Vc_box.value = ("c = " + str(np.around((V/V0)**(1/3)*c0,3))[1:-1] + " Å")
              
P_slider = widgets.FloatSlider(min=0, max=200, step=0.1, value=0, description='P in GPa:',slider_color='green',layout=Layout(width='52.5%'))            
widgets.interactive(V_calc,P=P_slider)

VV_box = widgets.Text(disabled=True,color='black',layout=Layout(width='12%'))
Va_box = widgets.Text(disabled=True,color='black',layout=Layout(width='12%'))
Vb_box = widgets.Text(disabled=True,color='black',layout=Layout(width='12%'))
Vc_box = widgets.Text(disabled=True,color='black',layout=Layout(width='12%'))
#box_titles = ['VV_box', 'Va_box','Vb_box','Vc_box']
#boxes = [widgets.Text(disabled=True,color='black',layout=Layout(width='12%')) for i in box_titles]

VBox([HBox([P_slider]), HBox([VV_box, Va_box, Vb_box, Vc_box])])